In [1]:
from datetime import datetime

In [2]:
import pickle
with open('Clean-dataset-1.pkl','rb') as fp:
    df = pickle.load(fp)
df.shape

(24053764, 4)

In [3]:
df = df.sort_values(by='Date')

df = df.drop(axis=1,labels=['Date'])

In [4]:
import scipy as np
CID = np.array(df.CustomerID)
MID = np.array(df.MovieID)
Score = np.array(df.Score)
l = int(df.shape[0]*0.7)

In [5]:
tr_data = {}
for i in range(l):
    r = CID[i]; c = MID[i]
    tr_data[(r,c)] = Score[i]

In [6]:
ts_data = {}
for i in range(l,CID.shape[0]):
    r = CID[i]; c = MID[i]
    ts_data[(r,c)] = Score[i]

In [7]:
row = CID.max()+1
col = MID.max()+1
del df,MID,CID,Score

In [8]:
import gc; gc.collect()

7

In [79]:
from datetime import datetime
p = datetime.now()
str(p)[11:]

'11:56:20.944381'

In [84]:
import numpy
from datetime import datetime
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.1, beta=0.02):
    Q = Q.T
    key_lst = list(R.keys())
    #print ('Entered -- ',str(datetime.now())[11:]) # Time of entering
    for step in range(steps):
        for tup in key_lst:
            i,j=tup
            eij = R[tup] - numpy.dot(P[i,:],Q[:,j])
            for k in range(K):
                P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        #print ('Exiting 1st chunk -- ',str(datetime.now())[11:]) # Time of exiting the 1st chunk
        e = 0
        for tup in key_lst:
            i,j=tup
            e = e + pow(R[tup] - numpy.dot(P[i,:],Q[:,j]), 2)
            for k in range(K):
                e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        #print ('Exiting 2nd chunk -- ',str(datetime.now())[11:]) # Time of exiting the 2nd chunk
        if e < 0.001:
            break
    #print ('Completion -- ',str(datetime.now())[11:]) # Time of completion
    return P, Q

In [85]:
R = tr_data
N = row
M = col
K = 100

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = numpy.dot(nP, nQ.T)

/home/sanjay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in double_scalars
  if sys.path[0] == '':
/home/sanjay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]


KeyboardInterrupt: 